# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [18]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-sport1-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series ("Association Football Players explorative search") 

Consider the following exploratory information need:

> investigate the association football players and find the main BGPs related to this sports. Compare awards and participation in the competitions of the players

**Important consideration: in wikidata, we are talking of ASSOCIATION FOOTBALL**

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q11571`| Cristiano Ronaldo       | node |





Also consider

```
wd:Q11571 ?p ?obj .
```

is the BGP to retrieve all **properties of Cristiano Ronaldo**


The workload should:


1. Identify the BGP for association football player

2. Identify the BGP for association football club

3. Identify the BGP for association football league

4. Which are the association football leagues in Italy?

5. Identify the BGP for the ballon d'Or award

6. Find the countries that have at least two players of their nationality who won the ballon d'Or

7. Find the football clubs with more players who played for that club that won the ballon d'Or

8. Explore ballon d'Or over years. Are there ways to find the year when a player won the ballon d'Or?

9. Who is the player who played in more different football clubs?

10. Identify the BGP of the FIFA World Cup

11. Who are the players who partecipated in more than 4 world's cup editions? (>=5)


## Task 1 
Identify the BGP for football player

In [19]:
# starting from Cristiano Ronaldo I look at his properties
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
    wd:Q11571 ?pr ?obj.
    ?pr <http://schema.org/name> ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('pr', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('pr', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('pr', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('pr', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('pr', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[

In [20]:
# starting from Cristiano Ronaldo I look at his occupations
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    wd:Q11571 wdt:P106 ?obj.
    ?obj <http://schema.org/name> ?name .
} 
LIMIT 100
"""

print("Results")
x = run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q937857'), ('name', 'association football player')]
1


`wd:Q93785` is the node of the association football player in wikidata

## Task 2
Identify the BGP for football team

In [21]:
# in the first query I found P54 that is member of sports team. Use it and explore
queryString = """
SELECT DISTINCT ?team ?name
WHERE { 
    wd:Q11571 wdt:P54 ?team .
    ?team <http://schema.org/name> ?name .
}
"""

print("Results")
x= run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q3590758'), ('name', 'Portugal Olympic football team')]
[('team', 'http://www.wikidata.org/entity/Q1422'), ('name', 'Juventus F.C.')]
[('team', 'http://www.wikidata.org/entity/Q18656'), ('name', 'Manchester United F.C.')]
[('team', 'http://www.wikidata.org/entity/Q8682'), ('name', 'Real Madrid CF')]
[('team', 'http://www.wikidata.org/entity/Q1630430'), ('name', 'Portugal national under-21 football team')]
[('team', 'http://www.wikidata.org/entity/Q267245'), ('name', 'Portugal national association football team')]
[('team', 'http://www.wikidata.org/entity/Q75729'), ('name', 'Sporting CP')]
[('team', 'http://www.wikidata.org/entity/Q1772776'), ('name', 'Portugal national under-20 football team')]
[('team', 'http://www.wikidata.org/entity/Q3590754'), ('name', 'Portugal national under-17 football team')]
[('team', 'http://www.wikidata.org/entity/Q21079208'), ('name', 'Portugal national under-15 football team')]
10


In [22]:
# look for the istance of the results found above
queryString = """
SELECT DISTINCT ?inst ?name
WHERE { 
    wd:Q11571 wdt:P54 ?team .
    ?team wdt:P31 ?inst.  
    ?inst <http://schema.org/name> ?name .
}
"""

print("Results")
x= run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q23759293'), ('name', 'national Olympic football team')]
[('inst', 'http://www.wikidata.org/entity/Q15944511'), ('name', 'association football team')]
[('inst', 'http://www.wikidata.org/entity/Q476028'), ('name', 'association football club')]
[('inst', 'http://www.wikidata.org/entity/Q4830453'), ('name', 'business')]
[('inst', 'http://www.wikidata.org/entity/Q891723'), ('name', 'public company')]
[('inst', 'http://www.wikidata.org/entity/Q6979593'), ('name', 'national association football team')]
[('inst', 'http://www.wikidata.org/entity/Q847017'), ('name', 'sports club')]
[('inst', 'http://www.wikidata.org/entity/Q23847779'), ('name', 'national under-23 association football team')]
[('inst', 'http://www.wikidata.org/entity/Q23904673'), ('name', 'national under-15 association football team')]
9


`wd:Q476028` is the node of the association football club in wikidata

## Task 3 
Identify the BGP for football league

In [23]:
# In the query above I found Real Madrid, wd:Q8682 look at its properties
queryString = """
SELECT DISTINCT ?p ?pName
WHERE { 

wd:Q8682 ?p ?obj.

?p <http://schema.org/name> ?pName .
}
"""

print("Results")
x = run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('pName', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('pName', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('pName', 'home venue')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('pName', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('pName', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('pName', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pName', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pName', 'headquarters location')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award 

In [24]:
# Found property league (wdt:P118). Let's see the result
queryString = """
SELECT DISTINCT ?league ?name
WHERE { 

wd:Q8682 wdt:P118 ?league.

?league <http://schema.org/name> ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('league', 'http://www.wikidata.org/entity/Q324867'), ('name', 'La Liga')]
1


In [25]:
# Real Madrid plays in La Liga. Find the instance.
queryString = """
SELECT DISTINCT ?inst ?name
WHERE { 

wd:Q8682 wdt:P118 ?league.
?league wdt:P31 ?inst.

?inst <http://schema.org/name> ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q15991290'), ('name', 'professional sports league')]
[('inst', 'http://www.wikidata.org/entity/Q15991303'), ('name', 'association football league')]
2


`wd:Q15991303` is the node of the association football league in wikidata

## Task 4
Which are the association football leagues in Italy?

We don't know the node Italy, but we know the property nationality that is wdt:P27. Starting from football players we can find their nationality.

In [26]:
# bind the football player and look at his nationality
queryString = """
SELECT DISTINCT ?nation ?name
WHERE { 

?player wdt:P106 wd:Q937857;
        wdt:P27 ?nation.

?nation sc:name ?name .
}
ORDER BY (?name)
"""

print("Results")
x = run_query(queryString)

Results
[('nation', 'http://www.wikidata.org/entity/Q13417403'), ('name', '.الجزائر')]
[('nation', 'http://www.wikidata.org/entity/Q889'), ('name', 'Afghanistan')]
[('nation', 'http://www.wikidata.org/entity/Q170366'), ('name', 'Ahwaz')]
[('nation', 'http://www.wikidata.org/entity/Q137014'), ('name', 'AlSaudiah')]
[('nation', 'http://www.wikidata.org/entity/Q222'), ('name', 'Albania')]
[('nation', 'http://www.wikidata.org/entity/Q1242155'), ('name', 'Albanians in Kosovo')]
[('nation', 'http://www.wikidata.org/entity/Q262'), ('name', 'Algeria')]
[('nation', 'http://www.wikidata.org/entity/Q218272'), ('name', 'Algeria')]
[('nation', 'http://www.wikidata.org/entity/Q2415901'), ('name', 'Allied-occupied Germany')]
[('nation', 'http://www.wikidata.org/entity/Q16641'), ('name', 'American Samoa')]
[('nation', 'http://www.wikidata.org/entity/Q228'), ('name', 'Andorra')]
[('nation', 'http://www.wikidata.org/entity/Q916'), ('name', 'Angola')]
[('nation', 'http://www.wikidata.org/entity/Q25228'),

`wd:Q38` is Italy

In [27]:
# look at properties of intsnces of football league
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
?league wdt:P31 wd:Q15991303;
        ?prop ?obj.

?prop sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P6118'), ('name', 'season ends')]
[('prop', 'http://www.wikidata.org/prop/direct/P7441'), ('name', 'WorldFootball.net competition ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P4003'), ('name', 'Facebook page ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('prop', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('prop', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('prop', 'http://www.wikidata.org/prop/direct/P1350'), ('name', 'number of matches played/races/starts')]
[('prop', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('prop', 'http://www.wikidata.org/prop/direct/P137'), ('name', 'operator')]
[('prop', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('prop', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('pr

In [28]:
# find the italian football league using the property country found above (P17)
queryString = """
SELECT DISTINCT ?league ?name
WHERE { 
?league wdt:P31 wd:Q15991303;
        wdt:P17 wd:Q38.

?league sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('league', 'http://www.wikidata.org/entity/Q645148'), ('name', 'Eccellenza')]
[('league', 'http://www.wikidata.org/entity/Q15804'), ('name', 'Serie A')]
[('league', 'http://www.wikidata.org/entity/Q607965'), ('name', 'Serie C')]
[('league', 'http://www.wikidata.org/entity/Q677397'), ('name', 'Lega Pro Prima Divisione')]
[('league', 'http://www.wikidata.org/entity/Q1141778'), ('name', 'Serie D')]
[('league', 'http://www.wikidata.org/entity/Q830717'), ('name', 'Lega Pro Seconda Divisione')]
[('league', 'http://www.wikidata.org/entity/Q1063016'), ('name', 'Prima Categoria')]
[('league', 'http://www.wikidata.org/entity/Q3717991'), ('name', 'Eccellenza Campania')]
[('league', 'http://www.wikidata.org/entity/Q3718091'), ('name', 'Eccellenza Liguria')]
[('league', 'http://www.wikidata.org/entity/Q3718140'), ('name', 'Eccellenza Marche')]
[('league', 'http://www.wikidata.org/entity/Q3718185'), ('name', 'Eccellenza Piedmont and Aosta Valley')]
[('league', 'http://www.wikidata.org/entit

## Task 5
Find BGP for the ballon d'Or award

In [29]:
# football players properties
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 

?player wdt:P106 wd:Q937857;
        ?prop ?obj.

?prop sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P4421'), ('name', 'Sportbox.ru ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P4262'), ('name', 'Footoféminin.fr player ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('prop', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('prop', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('prop', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('prop', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('prop', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('prop', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('prop', 'http://www.wikidata.org/prop/direct/P1350'), ('name', 'number of matches played/races/starts')]
[('prop', 'http://www

In [30]:
# see the object of the properties awards received
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 

?player wdt:P106 wd:Q937857;
        wdt:P166 ?obj.

?obj sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q318770'), ('name', 'Order of Merit of North Rhine-Westphalia')]
[('obj', 'http://www.wikidata.org/entity/Q4974237'), ('name', 'Bronze Bauhinia Star')]
[('obj', 'http://www.wikidata.org/entity/Q89491370'), ('name', 'Honda Sports Award for Soccer')]
[('obj', 'http://www.wikidata.org/entity/Q1588911'), ('name', 'Golden Boy')]
[('obj', 'http://www.wikidata.org/entity/Q1278513'), ('name', 'Belgian Ebony Shoe')]
[('obj', 'http://www.wikidata.org/entity/Q29023922'), ('name', 'Honoured Coach of Russia')]
[('obj', 'http://www.wikidata.org/entity/Q1998642'), ('name', 'Belgian professional football awards')]
[('obj', 'http://www.wikidata.org/entity/Q12201526'), ('name', 'Member of the Order of the British Empire')]
[('obj', 'http://www.wikidata.org/entity/Q19030539'), ('name', 'Medal of the Order "For Merit to the Fatherland" I\xa0class')]
[('obj', 'http://www.wikidata.org/entity/Q315026'), ('name', 'Silbernes Lorbeerblatt')]
[('obj', 'http://www.

`wd:Q166177` is the Ballon d'Or

## Task 6
Find the countries who have at least two players of their nationality who won a ballon d'Or

In [31]:
# find countries with more players who won the ballon d'Or
queryString = """
SELECT ?country ?name count(*) AS ?won
WHERE { 

?people wdt:P27 ?country;
        wdt:P106 wd:Q937857;
        wdt:P166 wd:Q166177.

?country sc:name ?name
}
GROUP BY ?country ?name
HAVING (count(*) > 1)
ORDER BY DESC (?won)
"""

print("Results")
x = run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy'), ('won', '6')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom'), ('won', '6')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany'), ('won', '5')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil'), ('won', '4')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('name', 'Spain'), ('won', '4')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('won', '4')]
[('country', 'http://www.wikidata.org/entity/Q212'), ('name', 'Ukraine'), ('won', '3')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('name', 'Argentina'), ('won', '3')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('name', 'Kingdom of the Netherlands'), ('won', '3')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('name', 'Portugal'), ('won', '3')]
[('country', 'http://www.wikidata.org/entity/Q15180'), ('name', 'Soviet Union'), ('won', '

## Task 7
Find the top-5 football clubs with more players who played for that club that won the ballon d'Or

In [32]:
# find clubs with more players who won the ballon d'Or
queryString = """
SELECT ?team ?name count(*) AS ?won
WHERE { 
?player wdt:P106 wd:Q937857;
        wdt:P54 ?team ;
        wdt:P106 wd:Q937857;
        wdt:P166 wd:Q166177.

?team sc:name ?name
}
GROUP BY ?team ?name
HAVING (count(*) > 1)
ORDER BY DESC (?won)
LIMIT 5
"""

print("Results")
x = run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q1543'), ('name', 'A.C. Milan'), ('won', '12')]
[('team', 'http://www.wikidata.org/entity/Q8682'), ('name', 'Real Madrid CF'), ('won', '10')]
[('team', 'http://www.wikidata.org/entity/Q1422'), ('name', 'Juventus F.C.'), ('won', '8')]
[('team', 'http://www.wikidata.org/entity/Q7156'), ('name', 'FC Barcelona'), ('won', '8')]
[('team', 'http://www.wikidata.org/entity/Q631'), ('name', 'FC Internazionale Milano'), ('won', '7')]
5


## Task 8
Explore ballon d'Or over years. Are there ways to find the year when a player won the ballon d'Or?

In [33]:
# I know ballon d'Or and football player, let's see some players who won it.
queryString = """
SELECT DISTINCT ?player ?name
WHERE { 

?player wdt:P106 wd:Q937857;
        wdt:P166 wd:Q166177.
?player sc:name ?name.
} 
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q162043'), ('name', 'Eusébio')]
[('player', 'http://www.wikidata.org/entity/Q128829'), ('name', 'Michael Owen')]
[('player', 'http://www.wikidata.org/entity/Q102027'), ('name', 'Fabio Cannavaro')]
[('player', 'http://www.wikidata.org/entity/Q36386'), ('name', 'Lothar Matthäus')]
[('player', 'http://www.wikidata.org/entity/Q178628'), ('name', 'Paolo Rossi')]
[('player', 'http://www.wikidata.org/entity/Q216140'), ('name', 'Stanley Matthews')]
[('player', 'http://www.wikidata.org/entity/Q310657'), ('name', 'Matías Fernandez')]
[('player', 'http://www.wikidata.org/entity/Q483145'), ('name', 'Luís Figo')]
[('player', 'http://www.wikidata.org/entity/Q483577'), ('name', 'Rivaldo')]
[('player', 'http://www.wikidata.org/entity/Q483837'), ('name', 'Luka Modrić')]
10


I do not have the information on the year when people won the ballon d'Or, so I will explore deeper the ballon d'Or 

In [34]:
# Explore ballon d'Or properties
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 

wd:Q166177 ?prop ?obj.
?prop sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('prop', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('prop', 'http://www.wikidata.org/prop/direct/P2572'), ('name', 'hashtag')]
[('prop', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('prop', 'http://www.wikidata.org/prop/direct/P2924'), ('name', 'Great Russian Encyclopedia Online ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('prop', 'http://www.wikidata.org/prop/direct/P3222'), ('name', 'NE.se ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('prop', 'http://www.wikidata.org/prop/direct/P571'), ('name', 'inception')]
[('prop', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('prop', '

In [35]:
# Explore ballon d'Or properties
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 

?obj  ?prop wd:Q166177.
?prop sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic')]
[('prop', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('prop', 'http://www.wikidata.org/prop/direct/P179'), ('name', 'part of the series')]
[('prop', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('prop', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('prop', 'http://www.wikidata.org/prop/direct/P3450'), ('name', 'sports season of league or competition')]
[('prop', 'http://www.wikidata.org/prop/direct/P971'), ('name', 'category combines topics')]
7


In [36]:
# Have a look of objects related with part of the series to ballon d'Or
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 

?obj wdt:P179 wd:Q166177.
?obj sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1973694'), ('name', "Ballon d'Or 2001")]
[('obj', 'http://www.wikidata.org/entity/Q2288534'), ('name', "Ballon d'Or 1960")]
[('obj', 'http://www.wikidata.org/entity/Q2291570'), ('name', "Ballon d'Or 1962")]
[('obj', 'http://www.wikidata.org/entity/Q427109'), ('name', "Ballon d'Or 1961")]
[('obj', 'http://www.wikidata.org/entity/Q2198524'), ('name', "Ballon d'Or 1975")]
[('obj', 'http://www.wikidata.org/entity/Q2304672'), ('name', "1977 Ballon d'Or")]
[('obj', 'http://www.wikidata.org/entity/Q2446218'), ('name', "1976 Ballon d'Or")]
[('obj', 'http://www.wikidata.org/entity/Q2455472'), ('name', "Ballon d'Or 2003")]
[('obj', 'http://www.wikidata.org/entity/Q2549500'), ('name', "Ballon d'Or 2002")]
[('obj', 'http://www.wikidata.org/entity/Q805304'), ('name', "Ballon d'Or 2004")]
[('obj', 'http://www.wikidata.org/entity/Q2372109'), ('name', "Ballon d'Or 1958")]
[('obj', 'http://www.wikidata.org/entity/Q2401580'), ('name', "Ballon d'Or 1959")

In [37]:
# Have a look of properties of objects related with part of the series to ballon d'Or
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 

?obj wdt:P179 wd:Q166177;
    ?prop ?x.
?prop sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('prop', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('prop', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('prop', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('prop', 'http://www.wikidata.org/prop/direct/P179'), ('name', 'part of the series')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('prop', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('prop', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('prop', 'http://www.wikidata.org/prop/direct/P580'), ('name', 'start time')]
[('prop', 'http://www.wikidata.org/prop/direct/P585'), ('name', 'point in time')]
[('prop', 'http://www.wikidata.org/

In [38]:
# Have a look of properties of objects related with part of the series to ballon d'Or
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 

?obj wdt:P179 wd:Q166177.
?x ?prop ?obj.
?prop sc:name ?name.

}
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('prop', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('prop', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('prop', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
4


In [39]:
# Have a look to the properties of objects related with part of the series to ballon d'Or
queryString = """
SELECT DISTINCT ?player ?name ?obj ?ballon
WHERE { 
?player wdt:P106 wd:Q937857;
        wdt:P166 ?obj.
?obj wdt:P179 wd:Q166177.

?player sc:name ?name.
?obj sc:name ?ballon.

}
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q270367'), ('name', 'Luis Suárez'), ('obj', 'http://www.wikidata.org/entity/Q2288534'), ('ballon', "Ballon d'Or 1960")]
[('player', 'http://www.wikidata.org/entity/Q167828'), ('name', 'Lev Yashin'), ('obj', 'http://www.wikidata.org/entity/Q2292330'), ('ballon', "Ballon d'Or 1963")]
2


The results are not interesting because only two players are related with the ballon d'Or on the specific year

## Task 9
Who is the player who played in more football clubs?

In [40]:
# print the awards won by the players
queryString = """
SELECT DISTINCT ?player ?name count(DISTINCT ?club) AS ?clubs
WHERE { 

?player wdt:P106 wd:Q937857;
        wdt:P54 ?club.

?player sc:name ?name .
}
ORDER BY DESC (?clubs)
LIMIT 5
"""

print("Results")
x = run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q714354'), ('name', 'Túlio Maravilha'), ('clubs', '33')]
[('player', 'http://www.wikidata.org/entity/Q6175529'), ('name', 'Jefferson Louis'), ('clubs', '32')]
[('player', 'http://www.wikidata.org/entity/Q7839060'), ('name', 'Trevor Benjamin'), ('clubs', '31')]
[('player', 'http://www.wikidata.org/entity/Q6224012'), ('name', 'John Burridge'), ('clubs', '30')]
[('player', 'http://www.wikidata.org/entity/Q461597'), ('name', 'Lutz Pfannenstiel'), ('clubs', '29')]
5


## Task 10
Identify the BGP of the FIFA World Cup

In [41]:
# print properties of the players that are not literals
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 

?player wdt:P106 wd:Q937857;
        ?prop ?x.
FILTER(!isLiteral(?x)).
?prop sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('prop', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('prop', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('prop', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('prop', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('prop', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('prop', 'http://www.wikidata.org/prop/direct/P140'), ('name', 'religion')]
[('prop', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('prop', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]
[('prop', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different fro

In [42]:
# print objects of the properties related with partecipant in (P1344)
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 

?player wdt:P106 wd:Q937857;
        wdt:P1344 ?x.
?x sc:name ?name .
}
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q104439579'), ('name', '2020 Belarusian Premier League relegation play-offs')]
[('x', 'http://www.wikidata.org/entity/Q77927099'), ('name', '2020 Belarusian Premier League')]
[('x', 'http://www.wikidata.org/entity/Q11831154'), ('name', '2019 Africa Cup of Nations')]
[('x', 'http://www.wikidata.org/entity/Q11865693'), ('name', 'football at the 2016 Summer Olympics')]
[('x', 'http://www.wikidata.org/entity/Q8150'), ('name', '1936 Summer Olympics')]
[('x', 'http://www.wikidata.org/entity/Q48432'), ('name', '1930 FIFA World Cup')]
[('x', 'http://www.wikidata.org/entity/Q18479838'), ('name', "football at the 2016 Summer Olympics – women's tournament")]
[('x', 'http://www.wikidata.org/entity/Q62589205'), ('name', "football at the 2020 Summer Olympics – women's tournament")]
[('x', 'http://www.wikidata.org/entity/Q102920'), ('name', 'UEFA Euro 2004')]
[('x', 'http://www.wikidata.org/entity/Q1185431'), ('name', '2015 Pan American Games')]
[('x', '

I found a lot of objects that seems to be part of a super class or something else. Starting point: `wd:Q37285`

In [43]:
# print objects of the properties related with partecipant in (P1344)
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 

wd:Q37285 ?prop ?x.
?prop sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('prop', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('prop', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('prop', 'http://www.wikidata.org/prop/direct/P1350'), ('name', 'number of matches played/races/starts')]
[('prop', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('prop', 'http://www.wikidata.org/prop/direct/P1546'), ('name', 'motto')]
[('prop', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('prop', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'L

In [44]:
# use the property sports league or competition
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 

wd:Q37285 wdt:P3450 ?x.
?x sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q19317'), ('name', 'FIFA World Cup')]
1


`wd:Q19317` is the FIFA World Cup 

## Task 11
Who are the players who partecipated in more than 4 world's cup editions? (>=5)

In [45]:
# print properties of the players that are not literals
queryString = """
SELECT DISTINCT ?player ?name count(*) AS ?wcs
WHERE { 

?player wdt:P106 wd:Q937857;
        wdt:P1344 ?wc.
?wc wdt:P3450 wd:Q19317.
?player sc:name ?name
}
GROUP BY ?player ?name
HAVING (count(*) > 4)
ORDER BY DESC (?wcs)
"""

print("Results")
x = run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q36386'), ('name', 'Lothar Matthäus'), ('wcs', '5')]
[('player', 'http://www.wikidata.org/entity/Q186330'), ('name', 'Rafael Márquez'), ('wcs', '5')]
[('player', 'http://www.wikidata.org/entity/Q265428'), ('name', 'Antonio Carbajal'), ('wcs', '5')]
[('player', 'http://www.wikidata.org/entity/Q68060'), ('name', 'Gianluigi Buffon'), ('wcs', '5')]
4


## Task 99 Other useful things for other worflows
wd:Q18756 UCL

In [46]:
# verify if there exists a connection between UCL editions and teams
queryString = """
SELECT DISTINCT ?prop ?name  
WHERE { 
?team wdt:P31 wd:Q476028.
?team ?prop ?x.


?prop sc:name ?name
}
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P4421'), ('name', 'Sportbox.ru ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue')]
[('prop', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('prop', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('prop', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('prop', 'http://www.wikidata.org/prop/direct/P1320'), ('name', 'OpenCorporates ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('prop', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('prop', 'http://www.wikidata.org/prop/direct/P1454'), ('name', 'legal form')]
[('prop', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('prop', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location')]
[('prop', 'http://www.wikidata.org/prop/direct/P17'), ('name

In [31]:
# find UCL properties
queryString = """
SELECT DISTINCT ?prop ?name 
WHERE { 
?x ?prop wd:Q18756.

?prop sc:name ?name.

}
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic')]
[('prop', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('prop', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('prop', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('prop', 'http://www.wikidata.org/prop/direct/P3450'), ('name', 'sports season of league or competition')]
[('prop', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('prop', 'http://www.wikidata.org/prop/direct/P921'), ('name', 'main subject')]
[('prop', 'http://www.wikidata.org/prop/direct/P971'), ('name', 'category combines topics')]
[('prop', 'http://www.wikidata.org/prop/direct/P2499'), ('name', 'league level above')]
[('prop', 'http://www.wikidata.org/prop/direct/P3085'), ('name', 'qualifies for event')]
10


In [32]:
# show objects related with P3450
queryString = """
SELECT DISTINCT ?x ?name 
WHERE { 
?x wdt:P3450 wd:Q18756.

?x sc:name ?name.

}
"""

print("Results")
x=run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q497052'), ('name', '1964–65 European Cup')]
[('x', 'http://www.wikidata.org/entity/Q500084'), ('name', '1963–64 European Cup')]
[('x', 'http://www.wikidata.org/entity/Q30608'), ('name', '2013–14 UEFA Champions League')]
[('x', 'http://www.wikidata.org/entity/Q13416183'), ('name', '2015–16 UEFA Champions League')]
[('x', 'http://www.wikidata.org/entity/Q269535'), ('name', '1991–92 European Cup')]
[('x', 'http://www.wikidata.org/entity/Q308996'), ('name', '2004–05 UEFA Champions League')]
[('x', 'http://www.wikidata.org/entity/Q696237'), ('name', '1988–89 European Cup')]
[('x', 'http://www.wikidata.org/entity/Q696245'), ('name', '1986–87 European Cup')]
[('x', 'http://www.wikidata.org/entity/Q219384'), ('name', '1955–56 European Cup')]
[('x', 'http://www.wikidata.org/entity/Q456097'), ('name', '2000–01 UEFA Champions League')]
[('x', 'http://www.wikidata.org/entity/Q18740961'), ('name', '2016–17 UEFA Champions League')]
[('x', 'http://www.w

In [33]:
# verify if there exists a connection between UCL editions and teams
queryString = """
SELECT DISTINCT ?prop ?name  
WHERE { 
?team wdt:P31 wd:Q476028.
?x wdt:P3450 wd:Q18756.
?team ?prop ?x.


?prop sc:name ?name
}
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('prop', 'http://www.wikidata.org/prop/direct/P2522'), ('name', 'victory')]
2


In [45]:
# show UCL edition with the winner club
queryString = """
SELECT ?team ?name count(?ucl) AS ?champions
WHERE { 
VALUES ?player {wd:Q615 wd:Q11571}.
?team wdt:P31 wd:Q476028.
?x wdt:P3450 wd:Q18756.
?team wdt:P2522 ?x.
?player wdt:P54 ?team.


?team sc:name ?name.
?x sc:name ?ucl.
}
GROUP BY ?team ?name
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q1422'), ('name', 'Juventus F.C.'), ('champions', '2')]
[('team', 'http://www.wikidata.org/entity/Q18656'), ('name', 'Manchester United F.C.'), ('champions', '2')]
[('team', 'http://www.wikidata.org/entity/Q7156'), ('name', 'FC Barcelona'), ('champions', '5')]
[('team', 'http://www.wikidata.org/entity/Q8682'), ('name', 'Real Madrid CF'), ('champions', '13')]
4
